# Working rechunking with AMPS



In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os

import gcsfs
from tqdm import tqdm
import fsspec
import xarray as xr


In [2]:
from dask.distributed import Client
import dask_gateway
gateway = dask_gateway.Gateway()
cluster = gateway.new_cluster()

In [3]:
cluster.scale(20)
client = Client(cluster)
cluster

### setup runtime parameters

In [4]:
gcsdir = 'gs://ldeo-glaciology'
ampsdir = 'AMPS'
amps_ver = 'WRF_24'

domain = 'domain_03'
filepattern = 'wrfout_d03_20190930*'
pattern = os.path.join(gcsdir, ampsdir, amps_ver, domain, filepattern)
print(pattern)

gs://ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_20190930*


### list the netcdf files

In [5]:
fs = gcsfs.GCSFileSystem(project='ldeo-glaciology', mode='ab', cache_timeout = 0)

In [8]:
NCs = fs.glob(pattern)
print(f"Total of {len(NCs)} wrf files.\n")
NCs

Total of 8 wrf files.



['ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093000_f003.nc',
 'ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093000_f006.nc',
 'ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093000_f009.nc',
 'ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093000_f012.nc',
 'ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093012_f003.nc',
 'ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093012_f006.nc',
 'ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093012_f009.nc',
 'ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093012_f012.nc']

### It can only handle 4 files at a time in my experience

In [9]:
NCs_urls = ['gs://' + x + '#mode=bytes' for x in NCs]
print(NCs_urls[0])


openfile = fs.open(NCs_urls[0], mode='rb') 
ds = xr.open_dataset(openfile, engine='h5netcdf',chunks={'south_north': -1, 
                                                               'west_east': -1,
                                                               'Time': -1})
for i in tqdm(range(1, 4)):#len(NCs_urls))):
    openfile = fs.open(NCs_urls[i], mode='rb') 
    temp = xr.open_dataset(openfile, engine='h5netcdf',chunks={'south_north': -1, 
                                                               'west_east': -1,
                                                               'Time': -1})
    ds = xr.concat([ds,temp],'Time')

print(f"Total size for {ds.DX[0]/1000:.2f} km output: {ds.nbytes / 1e9:.3f} Gb\n")        

gs://ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093000_f003.nc#mode=bytes


100%|██████████| 3/3 [00:04<00:00,  1.54s/it]

Total size for 2.67 km output: 10.722 Gb



### Remove some variables

In [12]:
vars_save = [
#     'U', 'V', 'W', 'PH', 'PHB', 'T', 'P', 'PB', 'P_HYD',
             'Q2', 'T2', 'PSFC', 'U10', 'V10',
#              'QVAPOR', 'QICE', 'QCLOUD', 'QRAIN', 'QSNOW', 
             'LANDMASK', 'HGT',
             'SST', 'SSTSK', 'TSK',
             'RAINC', 'RAINSH', 'RAINNC', 
             'QFX', 'HFX', 'LH', 'PBLH', 'ACSNOM', 'SWDOWN', 'LWDNB', 'GLW',
            'SNOW', 'SNOWH'
]

ds_stripped = ds[vars_save]
print(f'{ds_stripped.nbytes/1e9} Gb')
print(f'Originally {ds.nbytes/1e9} Gb')

0.279450032 Gb
Originally 10.721515004 Gb


## persist the result 

In [88]:
ds_stripped = ds_stripped.persist()

## Rechunk to group the 4 timesteps together



In [13]:
ds_stripped = ds_stripped.chunk({"Time": -1})

## persist the result

In [14]:
ds_stripped = ds_stripped.persist()

In [91]:
ds_stripped

<xarray.Dataset>
Dimensions:   (Time: 4, south_north: 1035, west_east: 675)
Coordinates:
    XLAT      (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    XLONG     (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    XTIME     (Time) datetime64[ns] dask.array<chunksize=(4,), meta=np.ndarray>
Dimensions without coordinates: Time, south_north, west_east
Data variables:
    Q2        (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    T2        (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    PSFC      (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    U10       (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    V10       (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    LANDMASK  (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    HGT       (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    SST       (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    SSTSK     (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    TSK       (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    RAINC     (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    RAINSH    (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    RAINNC    (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    QFX       (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    HFX       (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    LH        (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    PBLH      (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    ACSNOM    (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    SWDOWN    (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    LWDNB     (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    GLW       (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    SNOW      (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
    SNOWH     (Time, south_north, west_east) float32 dask.array<chunksize=(4, 1035, 675), meta=np.ndarray>
Attributes:
    TITLE:                            OUTPUT FROM WRF V3.9.1.1 MODEL
    START_DATE:                      2019-09-30_00:00:00
    SIMULATION_START_DATE:           2019-09-30_00:00:00
    WEST-EAST_GRID_DIMENSION:        [676]
    SOUTH-NORTH_GRID_DIMENSION:      [1036]
    BOTTOM-TOP_GRID_DIMENSION:       [61]
    DX:                              [2666.667]
    DY:                              [2666.667]
    SKEBS_ON:                        [0]
    SPEC_BDY_FINAL_MU:               [1]
    USE_Q_DIABATIC:                  [0]
    GRIDTYPE:                        C
    DIFF_OPT:                        [1]
    KM_OPT:                          [4]
    DAMP_OPT:                        [3]
    DAMPCOEF:                        [0.21]
    KHDIF:                           [0.]
    KVDIF:                           [0.]
    MP_PHYSICS:                      [4]
    RA_LW_PHYSICS:                   [4]
    RA_SW_PHYSICS:                   [2]
    SF_SFCLAY_PHYSICS:               [2]
    SF_SURFACE_PHYSICS:              [2]
    BL_PBL_PHYSICS:                  [2]


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/iostream.py", line 1391, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/srv/conda/envs/notebook/lib/python3.8/ssl.py", line 1309, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1124)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/asyncio/events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "/srv/conda/envs/notebook/lib/p

In [69]:
client.cancel(ds_stripped)

The lines below fail!

In [52]:
ds = ds.chunk({"Time": -1})


In [53]:
ds = ds.persist()

distributed.batched - INFO - Batched Comm Closed: in <closed TLS>: Stream is closed
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/site-packages/tornado/iostream.py", line 1391, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/srv/conda/envs/notebook/lib/python3.8/ssl.py", line 1309, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1124)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.8/asyncio/events.py", line 81, in _run
    s

In [40]:
ds.nbytes/1e9

10.721515004